In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'flash-datum-368412'

In [3]:
#packages
from google.cloud import storage
from google.cloud import bigquery

import pandas as pd
from sklearn import datasets

In [4]:
#clients
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

In [5]:
#parameters:
BUCKET = PROJECT_ID
BUCKET

'flash-datum-368412'

In [9]:
REGION = 'us-west1'

In [10]:
#Check to see if bucket already exist and create if missing:
if not gcs.lookup_bucket(BUCKET):
    bucketDef = gcs.bucket(BUCKET)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(f'Created Bucket: {gcs.lookup_bucket(BUCKET).name}')
else:
    bucketDef = gcs.bucket(BUCKET)
    print(f'Bucket already exist: {bucketDef.name}')

Created Bucket: flash-datum-368412


In [11]:
print(f'Review the storage bucket in the console here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID};tab=objects&project={PROJECT_ID}')


Review the storage bucket in the console here:
https://console.cloud.google.com/storage/browser/flash-datum-368412;tab=objects&project=flash-datum-368412


In [12]:
#Service Account & Permissions
#This notebook instance is running as a service account in GCP. This service account will also be used to run other services in Vertex AI like training jobs and pipelines. The service account will need permission to interact with object in Cloud Storage which requires the role (roles/storage.objectAdmin).

#Get the current service account:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'472530920109-compute@developer.gserviceaccount.com'

In [13]:
#Enable the Cloud Resource Manager API:
!gcloud services enable cloudresourcemanager.googleapis.com



Operation "operations/acat.p2-472530920109-1e371c61-1b51-47b8-8794-5a7c0e6fab30" finished successfully.


In [14]:
#List the service accounts current roles:

!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"


ROLE
roles/editor
roles/storage.objectAdmin


In [15]:
#If the resulting list is missing roles/storage.objectAdmin or another role that contains this permission, like the basic role roles/owner, then it will need to be added for the service account. Use these instructions to complete this:
print(f'Go To IAM in the Google Cloud Console:\nhttps://console.cloud.google.com/iam-admin/iam?orgonly=true&project={PROJECT_ID}&supportedpurview=organizationId')


Go To IAM in the Google Cloud Console:
https://console.cloud.google.com/iam-admin/iam?orgonly=true&project=flash-datum-368412&supportedpurview=organizationId


In [16]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"


ROLE
roles/editor
roles/storage.objectAdmin


In [17]:
#Update AIPlatform Package:
#The google-cloud-aiplatform package updates frequently. Update it for latest functionality.

#aiplatform Python Client
#GitHub Repo for api-common-protos
#For a better understanding of the Vertex AI APIs client, version, and layers please review the tip here aiplatform_notes.md.
!pip install googleapis-common-protos -U -q
!pip install google-cloud-aiplatform -U -q